In [2]:
# Ignore
from IPython.display import Image

## Q1. Write pseudocode for a parallel version of merge sort using a serial merge step. Compute the span and parallelism of your code. How useful is this code?

You may assume that the work is unchanged, $O(n\log n)$

```
merge_sort (A):
    if len(A) <= 1:
        return A
    len := len(A)
    first := A[:len/2]
    second := A[len/2:]

    first_sorted := spawn merge_sort(first)
    second_sorted := spawn merge_sort(second)
    sync

    return merge(first_sorted, second_sorted)
```

with

```
merge (first, second):
    out := empty array of len (len(first) + len(second))
    i, j := 0, 0
    for k from 0 to (len(first) + len(second)):
        if (j >= len(second)) || (first[i] <= second[j]):
            out[k] := first[i]
            i++
        else if (i >= len(first)) || (second[j] < first[i]):
            out[k] := second[j]
            j++
```

The span is

$$T_{\infty}(n) = max(T_{\infty}(n/2), T_{\infty}(n/2)) + O(n) = T_{\infty}(n/2) + O(n)$$

and with the master method we this since $log_2(1) = 0 < 1$:

$$T_{\infty}(n) = O(n)$$

which gives us a parallelism of

$$T_1/T_{\infty} = log(n)$$

## Q2. Write pseudocode for a parallel merge step, using the hints below. Compute its span.

If you're merging L and R, find the midpoint value x of L, and divide L and R into halves which are less than or greater than x. Use an efficient (serial) algorithm to find the midpoint of R. Find an upper bound of the span of the two spawned merge steps.

```
parallel_merge_sort A:
    if len(A) <= 1:
        return A
    len := len(A)
    first := A[:len/2]
    second := A[len/2:]

    first_sorted := spawn parallel_merge_sort(first)
    second_sorted := spawn parallel_merge_sort(second)
    sync

    return parallel_merge(first, second)

parallel_merge l, r:
    first_mid := l[len(l)/2]
    second_mid_idx := split_search(r, l[first_mid])
    first_l := l[:first_mid]
    first_r := l[first_mid+1:]
    second_l := r[:second_mid_idx]
    second_r := r[second_mid_idx:]
    
    left := spawn parallel_merge(first_l, second_l)
    right := spawn parallel_merge(first_r, second_r)
    sync

    return left + first[first_mid] + right

split_search A, x:
    if len(A) = 1:
        return 0 # always return some index
    mid :=len(A)/2
    if x > A[mid]:
        return mid + split_search(A[mid+1:])
    else if x == mid:
        return mid
    else
        return binary_search(A[:mid])
```

In [2]:
Image(url="parallel-merge-hint.png")


<div class="alert alert-block alert-info">
You may want to use the following variant of the master method:

If $T(n)=aT(n/b)+f(n)$ and $f(n)=\Theta(n^{\log_b a}(\log n)^k)$ then $T(n)=\Theta(n^{\log_b a}(\log n)^{k+1})$.
</div>

$$T_{\infty}(n) = T_{\infty}(n/2) + O(log n)$$

which gives us the above rule with $log_b(a) = 0$ and $k = 1$ to give:

$$T_{\infty}(n) = O((log n)^2)$$

## Q3. Write pseudocode for a fully parallel merge sort, and compute its span and parallelism. How useful is this code?

You may again assume the work is $O(n\log n)$.

See above for the pseudocode.

It's span is:

$$T_{\infty}(n) = T_{\infty}(n/2) + O((log n)^2)$$

which gives (using the same rule as above with $k = 2$):

$$T_{\infty}(n) = O((log n)^3)$$

This gives a parallelism of

$$T_1/T_{\infty} = n / (log n)^2$$

## Q4 (optional). Write code to implement this and verify that it works.

You don't need to actually use multiple processors, just verify that the code works.

In [47]:
def split_search(l, x):
    if len(l) == 1:
        return 0 if x < l[0] else 1
    if len(l) == 0:
        return 0
    mid = len(l)//2
    if x == l[mid]:
        return mid
    elif x < l[mid]:
        return split_search(l[:mid], x)
    else:
        return mid + 1 + split_search(l[mid+1:], x)

def parallel_merge(l, r):
    if len(l) < 1 or len(r) < 1:
        return l + r
    first_mid = len(l)//2
    second_mid = split_search(r, l[first_mid])
    first_l = l[:first_mid]
    first_r = l[first_mid+1:]
    second_l = r[:second_mid]
    second_r = r[second_mid:]
    
    left = parallel_merge(first_l, second_l)
    right = parallel_merge(first_r, second_r)

    return left + [l[first_mid]] + right

def parallel_merge_sort(l):
    if len(l) <= 1:
        return l
    mid = len(l)//2
    left = l[:mid]
    right = l[mid:]

    left_sorted = parallel_merge_sort(left)
    right_sorted = parallel_merge_sort(right)

    return parallel_merge(left_sorted, right_sorted)

In [49]:
parallel_merge_sort([10, 3, 1, 3, 4, 5, 200])

[1, 3, 3, 4, 5, 10, 200]